## Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [4]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

Загрузите датасет (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [2]:
path = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/5_tasks/3_natural_language_processing/data/3.4.2_smsspamcollection.txt'
data = pd.read_csv(path, sep='\t', header=None)
data.head()
corpus = data[1]
y = data[0] == 'spam'

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.toarray().shape, corpus.shape)

(5572, 8713) (5572,)


Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.


In [ ]:
np.random.seed(2)
lr = LogisticRegression()
cross_val_score(lr, X, y, cv=10, scoring='f1').mean()

0.9311542822856882

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

In [ ]:
lr.fit(X, y)
test_corpus = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
]
x_test = vectorizer.transform(test_corpus)
lr.predict(x_test)

array([ True,  True, False, False, False])

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [5]:
ngram_ranges = [(2, 2), (3, 3), (1, 3)]
for ngr in ngram_ranges:
    np.random.seed(2)
    pl = Pipeline([('cv', CountVectorizer(ngram_range=ngr)), 
                   ('lr', LogisticRegression())])
    print(cross_val_score(pl, corpus, y, cv=10, scoring='f1').mean())


0.8168951028736983
0.7249691484935827
0.9216545237836128


Повторите аналогичный эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [6]:
ngram_ranges = [(2, 2), (3, 3), (1, 3)]
for ngr in ngram_ranges:
    np.random.seed(2)
    x = CountVectorizer(ngram_range=ngr).fit_transform(corpus)
    print(cross_val_score(MultinomialNB(), x, y, scoring='f1').mean())

0.6545240488542623
0.39905005265073135
0.899621775059544


Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах?

Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [7]:
np.random.seed(2)
pl = Pipeline([('vectorizer', TfidfVectorizer()), 
                ('clf', LogisticRegression())])
print(cross_val_score(pl, corpus, y, cv=10, scoring='f1').mean())

0.8776671468367306


**Ответ: на наших данных качество понизилось.**